In [ ]:
import tensorflow as tf
import numpy as np
import gym
import math
from PIL import Image
import pygame, sys
from pygame.locals import *
from tensorflow import keras
from collections import deque
import random

In [ ]:
env = gym.make('MountainCar-v0')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
input_shape = env.observation_space.shape[0]
num_actions = env.action_space.n

In [ ]:
value_network = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(input_shape,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_actions)
])

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error")

In [ ]:
num_episodes = 1000
epsilon = 1
gamma = 0.9
state = env.reset()
batch = 200
replay = deque(maxlen=2000)
epoch = 0
alpha = 0.1

In [ ]:
for episode in range(num_episodes):
    state = env.reset()

    # Run the episode
    while True:

        value_function = value_network.predict(np.array([state]),verbose=0)[0]


        if np.random.rand()>epsilon:
            action = np.argmax(value_function)

        else:
            action = np.random.choice(num_actions)


        next_state, reward, done, _ = env.step(action)
        print(reward)

        done = 1 if done else 0

        replay.append((state,action,reward,next_state,done))

        state = next_state


        if done:
            break

        if len(replay)>batch:
            with tf.GradientTape() as tape:
                batch_ = random.sample(replay,batch)
                q_value1 = value_network(tf.convert_to_tensor([x[0] for x in batch_]))
                q_value2 = value_network(tf.convert_to_tensor([x[3] for x in batch_]))

                reward = tf.convert_to_tensor([x[2] for x in batch_])
                action = tf.convert_to_tensor([x[1] for x in batch_])
                done =   tf.convert_to_tensor([x[4] for x in batch_])


                actual_q_value1 = tf.cast(reward,tf.float64) + tf.cast(tf.constant(alpha),tf.float64)*(tf.cast(tf.constant(gamma),tf.float64)*tf.cast((tf.constant(1)-done),tf.float64)*tf.cast(tf.reduce_max(q_value2),tf.float64))
                loss = tf.cast(tf.gather(q_value1,action,axis=1,batch_dims=1),tf.float64)
                loss = loss - actual_q_value1
                loss = tf.reduce_mean(tf.math.pow(loss,2))


                grads = tape.gradient(loss, value_network.trainable_variables)
                optimizer.apply_gradients(zip(grads, value_network.trainable_variables))

                print('Epoch {} done with loss {} !!!!!!'.format(epoch,loss))

                value_network.save('keras/')
                if epoch%100==0:
                    epsilon*=0.999
                epoch+=1

In [ ]:
state = env.reset()
done = False
total_reward = 0

while True and not done:
  action = np.argmax(value_network.predict(np.array([state]))[0])
  next_state, reward, done, _ = env.step(action)
  state = next_state
  total_reward += reward

print(state)
print(total_reward)


1/1 [==============================] - 0s 20ms/step
[-0.79564524  0.00646731]
-200.0
